***GENERATED CODE FOR maxsalesdp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run maxsalesdpHooks.ipynb
try:
	#sourcePreExecutionHook()

	officemaxsalesdatatransfsa = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/OfficeMAX_Sales_DataTransfS1a.csv', 'filename': 'OfficeMAX_Sales_DataTransfS1a.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run maxsalesdpHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(officemaxsalesdatatransfsa,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/OfficeMAX_Sales_DataTransfS1a.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/OfficeMAX_Sales_DataTransfS1a.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": ["Order DateNew"], "FilePath": "/FileStore/platform/extra/657dfae2b6da8f8c082f82e11702911671/0part.csv", "requestRatio": 0.0, "totalRows": 100, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 13, "numberOfRows": 100, "duplicateRowCount": 0, "stats": [{"column": "Order Date", "alias": "Order Date", "generated": 0, "type": "String", "max": "9/1/2015", "min": "1/2/2015", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Order Date1", "alias": "Order Date1", "generated": 0, "type": "String", "max": "9/1/2015", "min": "1/2/2015", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Order DateNew", "alias": "Order DateNew", "generated": 0, "type": "date", "max": "2015211", "min": "201513", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Postal Code", "alias": "Postal Code", "generated": 0, "type": "numeric", "max": 98105, "min": 7036, "mean": 55145.18, "missing": 0.0, "stddev": 27139.45, "outliers": [], "validation": []}, {"column": "Region", "alias": "Region", "generated": 0, "type": "String", "max": "West", "min": "Central", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Category", "alias": "Category", "generated": 0, "type": "String", "max": "Technology", "min": "Furniture", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Sub-Category", "alias": "Sub-Category", "generated": 0, "type": "String", "max": "Tables", "min": "Accessories", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Cost", "alias": "Cost", "generated": 0, "type": "real", "max": 999.32, "min": 10.5, "mean": 498.1853, "missing": 0.0, "stddev": 317.37, "outliers": [], "validation": []}, {"column": "Price", "alias": "Price", "generated": 0, "type": "real", "max": 995.15, "min": 8.61, "mean": 541.3198, "missing": 0.0, "stddev": 281.42, "outliers": [], "validation": []}, {"column": "Profit", "alias": "Profit", "generated": 0, "type": "real", "max": 885.59, "min": -858.0, "mean": 43.133700000000005, "missing": 0.0, "stddev": 437.01, "outliers": [], "validation": []}, {"column": "Quantity", "alias": "Quantity", "generated": 0, "type": "numeric", "max": 25, "min": 1, "mean": 11.4, "missing": 0.0, "stddev": 7.02, "outliers": [], "validation": []}, {"column": "Sales", "alias": "Sales", "generated": 0, "type": "real", "max": 24607.22089, "min": 34.44196991, "mean": 6421.040881265101, "missing": 0.0, "stddev": 5764.1, "outliers": [19903.0828, 21148.32874, 22225.49018, 22944.36283, 24607.22089], "validation": []}, {"column": "CostPrice", "alias": "CostPrice", "generated": 1, "type": "real", "max": 1875.79, "min": 150.44, "mean": 1039.5050999999999, "missing": 0.0, "stddev": 410.94, "outliers": [], "validation": []}]}, "predictionPowerScore": [{"Category": 1.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 1.0}, {"Category": 0.0, "Cost": 1.0, "CostPrice": 0.2170188165, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.143531446, "Quantity": 0.0406832154, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0947051087, "CostPrice": 1.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0619953051, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 1.0, "Order Date1": 0.8683788122, "Order DateNew": 0.8508649902, "Postal Code": 0.7183520599, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.8764044944, "Order Date1": 1.0, "Order DateNew": 0.8508649902, "Postal Code": 0.7183520599, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 1.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0196733394, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.2146169057, "Order Date1": 0.2153843104, "Order DateNew": 0.3303500786, "Postal Code": 1.0, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.536748174, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0685942486, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 1.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.1727944105, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.130395782, "Profit": 1.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.0, "Quantity": 1.0, "Region": 0.0, "Sales": 0.1416666667, "Sub-Category": 0.0}, {"Category": 0.0, "Cost": 0.0574046555, "CostPrice": 0.0, "Order Date": 0.4640860607, "Order Date1": 0.4640860607, "Order DateNew": 0.4565061022, "Postal Code": 1.0, "Price": 0.0, "Profit": 0.1342494174, "Quantity": 0.0404109589, "Region": 1.0, "Sales": 0.0, "Sub-Category": 0.0765987197}, {"Category": 0.0, "Cost": 0.0, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.0, "Quantity": 0.2559337768, "Region": 0.0, "Sales": 1.0, "Sub-Category": 0.0}, {"Category": 0.1158823529, "Cost": 0.0246541694, "CostPrice": 0.0, "Order Date": 0.0, "Order Date1": 0.0, "Order DateNew": 0.0, "Postal Code": 0.0, "Price": 0.0, "Profit": 0.0, "Quantity": 0.0, "Region": 0.0, "Sales": 0.0, "Sub-Category": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "Cost", "type": "real", "min": "10.5", "max": "999.3", "mean": "317.4"}, {"columnName": "Price", "type": "real", "min": "8.6", "max": "995.1", "mean": "281.4"}], "functionsData": [{"operand1": "Cost", "operator": "add", "operand2": "Price", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Cost", "type": "real", "min": "10.5", "max": "999.3", "mean": "317.4"}}, {"Columndata": {"columnName": "Price", "type": "real", "min": "8.6", "max": "995.1", "mean": "281.4"}}, {"Columndata": ""}], "newColumnName": "CostPrice"}]}], "functionChanges": [{"columnName": "Cost", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "Cost", "operator": "add", "operand2": "Price", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Cost", "type": "real", "min": "10.5", "max": "999.3", "mean": "317.4"}}, {"Columndata": {"columnName": "Price", "type": "real", "min": "8.6", "max": "995.1", "mean": "281.4"}}, {"Columndata": ""}], "newColumnName": "CostPrice"}]}, {"columnName": "Price", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "Cost", "operator": "add", "operand2": "Price", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Cost", "type": "real", "min": "10.5", "max": "999.3", "mean": "317.4"}}, {"Columndata": {"columnName": "Price", "type": "real", "min": "8.6", "max": "995.1", "mean": "281.4"}}, {"Columndata": ""}], "newColumnName": "CostPrice"}]}], "fileheader": [{"field": "Order Date", "alias": "Order Date", "generated": 0, "position": 1, "type": "String"}, {"field": "Order Date1", "alias": "Order Date1", "generated": 0, "position": 2, "type": "String"}, {"field": "Order DateNew", "alias": "Order DateNew", "generated": 0, "position": 3, "type": "date"}, {"field": "Postal Code", "alias": "Postal Code", "generated": 0, "position": 4, "type": "numeric"}, {"field": "Region", "alias": "Region", "generated": 0, "position": 5, "type": "String"}, {"field": "Category", "alias": "Category", "generated": 0, "position": 6, "type": "String"}, {"field": "Sub-Category", "alias": "Sub-Category", "generated": 0, "position": 7, "type": "String"}, {"field": "Cost", "alias": "Cost", "generated": 0, "position": 8, "type": "real"}, {"field": "Price", "alias": "Price", "generated": 0, "position": 9, "type": "real"}, {"field": "Profit", "alias": "Profit", "generated": 0, "position": 10, "type": "real"}, {"field": "Quantity", "alias": "Quantity", "generated": 0, "position": 11, "type": "numeric"}, {"field": "Sales", "alias": "Sales", "generated": 0, "position": 12, "type": "real"}, {"field": "CostPrice", "alias": "CostPrice", "generated": 1, "position": 13, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run maxsalesdpHooks.ipynb
try:
	#sinkPreExecutionHook()

	sanitytest = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/658054c22e320b2314dd9df0/part-00000-4f094e83-5b49-4a3c-a5ce-5da6e4761bbb-c000.csv', 'samplecount': 100, 'originalcount': 100, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/OfficeMAX_Sales_DataTransfS1a.csv', 'pathOnly': '/Sanitytest', 'project_id': '64dc8260892b8d030d7486ef', 'parent_id': '64dc8260892b8d030d7486ef', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order Date', 'alias': 'Order Date', 'type': 'date', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order Date1', 'alias': 'Order Date1', 'type': 'date', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order DateNew', 'alias': 'Order DateNew', 'type': 'date', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Region', 'alias': 'Region', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Category', 'alias': 'Category', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sub-Category', 'alias': 'Sub-Category', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Cost', 'alias': 'Cost', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Price', 'alias': 'Price', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Profit', 'alias': 'Profit', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Quantity', 'alias': 'Quantity', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sales', 'alias': 'Sales', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'CostPrice', 'alias': 'CostPrice', 'type': 'real', 'position': '13'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order Date', 'alias': 'Order Date', 'type': 'date', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order Date1', 'alias': 'Order Date1', 'type': 'date', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Order DateNew', 'alias': 'Order DateNew', 'type': 'date', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Region', 'alias': 'Region', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Category', 'alias': 'Category', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sub-Category', 'alias': 'Sub-Category', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Cost', 'alias': 'Cost', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Price', 'alias': 'Price', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Profit', 'alias': 'Profit', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Quantity', 'alias': 'Quantity', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Sales', 'alias': 'Sales', 'type': 'real', 'position': '11'}], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'OfficeMaxTransformedDate.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-3f8f0ea2-ca71-4598-9911-f9e472145fe8-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
